In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from data.dataloader import AthenaLoader
from data.processing.processing import get_custom_data_from_db

In [ ]:
import pandas as pd

In [ ]:
loader = AthenaLoader()

In [ ]:
dataframes = loader.load_data()

In [ ]:
df_testing = dataframes['testing_summary']
del df_testing['partition_0']
del df_testing['new']

In [ ]:
df_testing = df_testing.loc[df_testing['district'] == 'mumbai', :]

In [ ]:
df_testing.dropna(axis=0, how='any', inplace=True)

In [ ]:
df_result = dataframes['new_covid_case_summary']
df_result = df_result.loc[:, :'deceased']
df_result['state'] = 'maharashtra'
df_result.dropna(axis=0, how='any', inplace=True)
df_result['date'] = pd.to_datetime(df_result['date'])

In [ ]:
get_custom_data_from_db(state='Maharashtra', district='Mumbai')